In [ ]:
import pandas as pd
import numpy as np

import re

#webscraping
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions,Chrome
from bs4 import BeautifulSoup
from undetected_chromedriver import ChromeOptions
import requests
import undetected_chromedriver as uc

In [ ]:
def attraction_images(URL):
    
    error = 0
     # Set up the Selenium WebDriver
    options = uc.ChromeOptions()
    options.add_argument("window-size=1200x800")
    options.add_argument('--user-data-dir=/Users/yasirutj/Library/Application Support/Google/Chrome/Profile 1')
    driver = uc.Chrome(options=options)
    driver.implicitly_wait(20)
    
    driver.get(URL)
    
    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1
    
    try:
        address_element = driver.find_element(By.XPATH, "//button[@class='UikNM _G B- _S _W _T c G_ wSSLS wnNQG raEkE']")#biGQs _P XWJSj Wb
        # print(address_element)
        address = address_element.text
    except:
        address = ""
    
    try:
        # click All photos button
        # use double quotes inside contains(text(), "All photos")
        all_photos_button = driver.find_element(By.XPATH, "//*[contains(text(), 'All photos')]")
        all_photos_button.click()
    except:
        error + 1
   
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")
    time.sleep(3)
    
    images = soup.find_all('img') #find all imgs
    
    #loop through imgs, check for 2 image types, add to list if it is a TripAdvisor user uploaded photo
    img_links = []
    errors = 0
    for i in images:
        try:
                link = i.attrs['src']
                if re.search('photo',link):
                    img_links.append(link)
        except:
            errors = errors + 1
            
            
    print(img_links)
    print(len(img_links))
    
    driver.quit()
    
    return address, img_links


In [ ]:
def attraction_base(url):
    
    attraction_df = pd.DataFrame(columns=['name', 'location', 'state', 'img_num'])
    idx = 1
    error = 0
    img_link_dict = {}
    
    if url == 'https://www.tripadvisor.co.uk/Attractions-g186217-Activities-oa0-England.html' :
        state = 'England'
    elif url == 'https://www.tripadvisor.co.uk/Attractions-g186425-Activities-oa0-Wales.html':
        state = 'Wales'
    elif url == 'https://www.tripadvisor.co.uk/Attractions-g186485-Activities-oa0-Scotland.html':
        state = 'Scotland'
    elif url == 'https://www.tripadvisor.co.uk/Attractions-g186469-Activities-oa0-Northern_Ireland.html':
        state = 'Nothern Ireland'
    
    # Set up the Selenium WebDriver
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    driver.implicitly_wait(20)  # Adjust the wait time as needed

    index = 0
    # Specify the URL for the 4 different states in UK
    driver.get(url)

    # Wait for any initial popups to load
    time.sleep(5)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    # Initialize BeautifulSoup with the current page source
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")

    url = "https://www.tripadvisor.com"
    links = {}
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page
        
    #loop through range to get numbers 1-30, we want top 30 attractions of each state, then for each number
    #loop through links, add attraction name and link to links dictionary if number match outside loop
    for x in range(1,31):
            find = str(x) + "[.] "
            for l in link_list:
                if re.search(find,l.text):
                    name = l.text
                    name = re.sub(r'^.*? ', '', name) #clean name
                    links[name] = url + l['href']
    
    next_page_btn = driver.find_element(By.XPATH, "//a[@class='BrOJk u j z _F wSSLS tIqAi unMkR']") #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 31-60,
    for x in range(31,61): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
    next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi iNBVo
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 61-90,
    for x in range(61,90): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
                
    next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 91-120,
    for x in range(91,120): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
        next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 121-150,
    for x in range(121,150): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
    
        next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 151-180,
    for x in range(151,180): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
        next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 181-210,
    for x in range(181,210): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
        next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 211-240,
    for x in range(211,240): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
    next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 241-270,
    for x in range(241,270): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 
                
        next_page_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next page"]') #BrOJk u j z _F wSSLS tIqAi unMkR
    next_page_btn.click()
    time.sleep(10)

    try:
        # Click "I Accept" button that shows up when the web page is first loaded
        i_accept = driver.find_element("id", "onetrust-accept-btn-handler").click()
    except:
        error + 1

    page_source = driver.page_source
    soup = BeautifulSoup(page_source) # specify lxml? not specified cus beautifoulsoup by default finds the parser that best suits our data type
    body = soup.find_all('body')[0]
    link_list = body.find_all( 'a') #find all links on page

    #loop through range to get numbers 91-120,
    for x in range(271,300): 
        find = re.compile(fr"{x}\. ")  # Using regex for the pattern
        for l in link_list:
            if find.search(l.text):
                name = re.sub(r'^.*? ', '', l.text)  # Clean name
                links[name] = url + l.get('href') 

    for name, link in links.items():
        address, img_links = attraction_images(link)
        img_link_dict[name] = img_links
        new_row = pd.DataFrame({'name':name, 'location': address, 'state': state , 'img_num' : len(img_links)}, index=[idx])
        idx += 1
        attraction_df = pd.concat ([attraction_df,new_row])
        
    # Close the WebDriver   
    driver.quit()
    
    return (attraction_df,img_link_dict)


In [ ]:
loc_df = pd.DataFrame(columns = ['name', 'location', 'img_num'])
img_dict = {}

In [ ]:
states = ['https://www.tripadvisor.co.uk/Attractions-g186217-Activities-oa0-England.html', 'https://www.tripadvisor.co.uk/Attractions-g186425-Activities-oa0-Wales.html', 'https://www.tripadvisor.co.uk/Attractions-g186485-Activities-oa0-Scotland.html', 'https://www.tripadvisor.co.uk/Attractions-g186469-Activities-oa0-Northern_Ireland.html']

for url in states:
    attraction_dataframe, images = attraction_base(url)

    loc_df = pd.concat([loc_df,attraction_dataframe], axis=0, ignore_index=True)
    img_dict.update(images)

loc_df.info()

In [ ]:
loc_df.to_pickle('attractions_data_all.pkl')

loc_df.to_csv("attractions_data_all.csv")

In [ ]:
img_dataframe = pd.DataFrame.from_dict(img_dict, orient= 'index')
img_dataframe.to_pickle('attractions_image_links_all.pkl')